# Data Processing

Now that I have collected data on country economic indicators and happiness data for countries in the Data Collection notebook, it is very important to clean the data, prepare it, and have it ready for use to analyze it. In this section, I took the raw World Bank JSON data and transformed it into a ready-to-use, structured tabular format. I exported this data into both a csv, and created a dataset, and exported that dataset into database.db. In addition to this, I compiled the 2018-2023 World Happiness Index data, pulled the columns I needed, and cleaned it as well, putting it into a structured dataset and csv, and exported it to database.db as well. 

An Overview of the pipeline is below:
- Converted World Bank JSON data for the selected economic indicators
- Cleaned column names, handled missing values, and calculated GDP per capita
- Formatted figures for readability
- Compiled World Happiness Reports from 2018–2023 into a single dataset
- Selected consistent metrics across years (happiness score, generosity, etc).
- Filtered to the set of countries that I am working with and added a year column for tracking.
The final outputs were structured, analysis-ready datasets that I stored in both CSV files and database.db.

## World Bank Data Cleaning and Processing

In [1]:
# Imported all of the imports that I need
import json
import pandas as pd
import os

# Imported my script and util python file
import sys
sys.path.append('/files/me204-2025-project-jayden/scripts')

# Imported util to reduce redundancy
from utils import indicators
from utils import year_column_map

import sqlite3
# Making sure to import SQL imports for database operations
from sqlalchemy import create_engine, text

After importing the right packages and utils, I loaded in the World Bank Data JSON in from the raw folder to clean and process it by converting it into a dataframe.

In [2]:
# Loaded the raw JSON data in
with open('/files/me204-2025-project-jayden/data/raw/world_bank_raw_data.json', 'r') as f:
    raw_json_data = json.load(f)

# Defined a function that parsed the JSON structure into a list of records (one per country-year-indicator)
# Each record represented a single data point for a specific country and year
# The function extracted the country name, ISO3 code, year, and the indicator value
# It also handled missing values by assigning None when the indicator value was not available
def parse_json_to_records(json_data, code):
    return [
        {
            'country': entry['country']['value'],
            # Extracted the full country name
            # iso3 is the code name (or sort of like an id) for each country
            'iso3': entry['country']['id'],
            # Extracted the ISO3 country code
            'year': int(entry['date']),
            # Converted the year from string to integer
            code: float(entry['value']) if entry['value'] is not None else None
        }
        for entry in json_data[1]
        # Iterated over the list of data entries in the JSON response
    ]


In [3]:
# Created a list of DataFrame (one for each indicator) using list comprehension
# Each DataFrame was renamed to use an easily readable label instead of the raw indicator code
dfs = [
    pd.DataFrame(parse_json_to_records(raw_json_data[code], code)).rename(columns={code: label})
    for code, label in indicators.items()
]

# Merged all individual DataFrames into a single DataFrame by iteratively joining on country, iso3, and year
df_full = dfs[0]
for df in dfs[1:]:
    df_full = pd.merge(df_full, df, on=['country', 'iso3', 'year'], how='outer')

# Sorted the final DataFrame by country code and year, then reset the index for cleanliness
df_full = df_full.sort_values(['iso3', 'year']).reset_index(drop=True)

# Displayed the final DataFrame
df_full

,country,iso3,year,GDP_Current_USD,Health_Expenditure_%_GDP,Education_Expenditure_%_GDP,Unemployment_Rate_%,Inflation_Annual_%,FDI_Net_Inflows_%_GDP,Population_Total,Life_Expectancy_Years,Govt_Effectiveness,Rule_of_Law,Control_of_Corruption,Political_Stability,Voice_and_Accountability
0,Denmark,DK,2018,3.552934e+11,10.097642,7.296910,5.131,0.813609,2.466701,5793636.0,80.953659,1.810602,1.773226,2.156996,0.931982,1.574022
1,Denmark,DK,2019,3.454015e+11,10.152435,7.255050,5.018,0.758132,-1.100671,5814422.0,81.451220,1.873268,1.833377,2.121649,0.967599,1.543434
2,Denmark,DK,2020,3.556310e+11,10.718546,7.383540,5.637,0.420712,0.363875,5831404.0,81.602439,1.840067,1.806794,2.236486,0.920234,1.514785
3,Denmark,DK,2021,4.083782e+11,10.747286,6.999930,5.043,1.853045,4.162143,5856733.0,81.404878,1.959734,1.895721,2.333371,0.928663,1.544976
4,Denmark,DK,2022,4.019456e+11,9.475862,5.296434,4.434,7.696567,7.667934,5903037.0,81.304878,1.990135,1.899581,2.402638,0.868918,1.591254
5,Denmark,DK,2023,4.070919e+11,9.417846,NaN,5.094,3.305178,1.124286,5946952.0,81.853659,2.015649,1.908936,2.376053,0.850848,1.664699
6,Denmark,DK,2024,4.294574e+11,NaN,NaN,5.587,1.372200,4.214609,5976992.0,NaN,NaN,NaN,NaN,NaN,NaN
7,Finland,FI,2018,2.738693e+11,9.045142,6.275580,7.361,1.083821,-3.844567,5515525.0,81.734146,2.013878,2.034399,2.171759,0.891890,1.589352
8,Finland,FI,2019,2.670148e+11,9.171427,6.416950,6.695,1.024094,6.097077,5521606.0,81.982927,1.972805,2.012629,2.113933,0.835259,1.562998
9,Finland,FI,2020,2.700003e+11,9.632496,6.634940,7.759,0.290555,-0.939114,5529543.0,81.931707,1.901004,2.018592,2.170440,0.981825,1.609512


After organizing the data into a dataframe, there seemed to be a large amount of columns and rows that need to be cleaned for easier readability. For example, columns are camel cased, values have a large amount of unnecessary decimals, GDP is in scientific notation, and the Country Code is not really needed. Lastly, 2024 has mostly NaN values, most likely because data hasn't been fully reported. I fixed these problems and cleaned the dataset below by dropping that year

In [4]:
# Round all numeric columns to 2 decimal places
df_full = df_full.round(2)

# Format the 'GDP_Current_USD' column without scientific notation using list comprehension
# Created a new list called 'formatted_gdp', and for each value in the 'GDP_Current_USD' column, format it to:
#   - include commas for thousands
#   - show exactly two decimal places
# If the value is null, keep it as None
# This ensures that GDP values are more readable and consistent instead of being in scientific notation
formatted_gdp = [
    float(val) if pd.notnull(val) else None
    # Formatted string
    for val in df_full["GDP_Current_USD"].values
    # Looping through all values in GDP
]
df_full["GDP_Current_USD"] = formatted_gdp

# Remove all rows where the 'year' column equals 2024
# This filters a lot of null values out
df_full = df_full[df_full['year'] < 2024].reset_index(drop=True)

df_full = df_full.drop('iso3', axis = 1)

df_full.head()

,country,year,GDP_Current_USD,Health_Expenditure_%_GDP,Education_Expenditure_%_GDP,Unemployment_Rate_%,Inflation_Annual_%,FDI_Net_Inflows_%_GDP,Population_Total,Life_Expectancy_Years,Govt_Effectiveness,Rule_of_Law,Control_of_Corruption,Political_Stability,Voice_and_Accountability
0,Denmark,2018,3.552934e+11,10.10,7.30,5.13,0.81,2.47,5793636.0,80.95,1.81,1.77,2.16,0.93,1.57
1,Denmark,2019,3.454015e+11,10.15,7.26,5.02,0.76,-1.10,5814422.0,81.45,1.87,1.83,2.12,0.97,1.54
2,Denmark,2020,3.556310e+11,10.72,7.38,5.64,0.42,0.36,5831404.0,81.60,1.84,1.81,2.24,0.92,1.51
3,Denmark,2021,4.083782e+11,10.75,7.00,5.04,1.85,4.16,5856733.0,81.40,1.96,1.90,2.33,0.93,1.54
4,Denmark,2022,4.019456e+11,9.48,5.30,4.43,7.70,7.67,5903037.0,81.30,1.99,1.90,2.40,0.87,1.59


In [5]:
# Reformat column names to improve readability
# Made country and year have the correct uppercase titles
df_full = df_full.rename(columns={"country": "Country", "year": "Year"})

# Replace all underscores "_" with spaces " " across all column headers
df_full.columns = [col.replace('_', ' ') for col in df_full.columns]
df_full.head()

,Country,Year,GDP Current USD,Health Expenditure % GDP,Education Expenditure % GDP,Unemployment Rate %,Inflation Annual %,FDI Net Inflows % GDP,Population Total,Life Expectancy Years,Govt Effectiveness,Rule of Law,Control of Corruption,Political Stability,Voice and Accountability
0,Denmark,2018,3.552934e+11,10.10,7.30,5.13,0.81,2.47,5793636.0,80.95,1.81,1.77,2.16,0.93,1.57
1,Denmark,2019,3.454015e+11,10.15,7.26,5.02,0.76,-1.10,5814422.0,81.45,1.87,1.83,2.12,0.97,1.54
2,Denmark,2020,3.556310e+11,10.72,7.38,5.64,0.42,0.36,5831404.0,81.60,1.84,1.81,2.24,0.92,1.51
3,Denmark,2021,4.083782e+11,10.75,7.00,5.04,1.85,4.16,5856733.0,81.40,1.96,1.90,2.33,0.93,1.54
4,Denmark,2022,4.019456e+11,9.48,5.30,4.43,7.70,7.67,5903037.0,81.30,1.99,1.90,2.40,0.87,1.59


After that portion of the cleaning, I noticed that GDP and population total has decimals that are not needed, so I'll change that below.

In [6]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Country                      36 non-null     object 
 1   Year                         36 non-null     int64  
 2   GDP Current USD              36 non-null     float64
 3   Health Expenditure % GDP     33 non-null     float64
 4   Education Expenditure % GDP  27 non-null     float64
 5   Unemployment Rate %          36 non-null     float64
 6   Inflation Annual %           36 non-null     float64
 7   FDI Net Inflows % GDP        36 non-null     float64
 8   Population Total             36 non-null     float64
 9   Life Expectancy Years        36 non-null     float64
 10  Govt Effectiveness           36 non-null     float64
 11  Rule of Law                  36 non-null     float64
 12  Control of Corruption        36 non-null     float64
 13  Political Stability   

It looks like GDP is a float. So it should be converted into an Int and rounded.

In [7]:
# Rounded 'Population_Total' to whole numbers by converting to integer
# Used pd.Series.round(0) and then converted it to int to remove decimals cleanly
df_full['Population Total'] = df_full['Population Total'].round(0).astype('Int64')

# Used the same logic
df_full['GDP Current USD'] = df_full['GDP Current USD'].round(0).astype('Int64')

df_full.head()

,Country,Year,GDP Current USD,Health Expenditure % GDP,Education Expenditure % GDP,Unemployment Rate %,Inflation Annual %,FDI Net Inflows % GDP,Population Total,Life Expectancy Years,Govt Effectiveness,Rule of Law,Control of Corruption,Political Stability,Voice and Accountability
0,Denmark,2018,355293374912,10.10,7.30,5.13,0.81,2.47,5793636,80.95,1.81,1.77,2.16,0.93,1.57
1,Denmark,2019,345401473013,10.15,7.26,5.02,0.76,-1.10,5814422,81.45,1.87,1.83,2.12,0.97,1.54
2,Denmark,2020,355631021932,10.72,7.38,5.64,0.42,0.36,5831404,81.60,1.84,1.81,2.24,0.92,1.51
3,Denmark,2021,408378204640,10.75,7.00,5.04,1.85,4.16,5856733,81.40,1.96,1.90,2.33,0.93,1.54
4,Denmark,2022,401945576108,9.48,5.30,4.43,7.70,7.67,5903037,81.30,1.99,1.90,2.40,0.87,1.59


I would also like to add the GDP per capita column, because it would be very useful in the future for more of a relative number to take away biasness for larger countries. It provided a normalized economic indicator no matter the country size.

In [8]:
# GDP / Population = GDP per Capita
df_full['GDP Per Capita'] = df_full['GDP Current USD'] / df_full['Population Total']
df_full.head()

,Country,Year,GDP Current USD,Health Expenditure % GDP,Education Expenditure % GDP,Unemployment Rate %,Inflation Annual %,FDI Net Inflows % GDP,Population Total,Life Expectancy Years,Govt Effectiveness,Rule of Law,Control of Corruption,Political Stability,Voice and Accountability,GDP Per Capita
0,Denmark,2018,355293374912,10.10,7.30,5.13,0.81,2.47,5793636,80.95,1.81,1.77,2.16,0.93,1.57,61324.766504
1,Denmark,2019,345401473013,10.15,7.26,5.02,0.76,-1.10,5814422,81.45,1.87,1.83,2.12,0.97,1.54,59404.266325
2,Denmark,2020,355631021932,10.72,7.38,5.64,0.42,0.36,5831404,81.60,1.84,1.81,2.24,0.92,1.51,60985.48856
3,Denmark,2021,408378204640,10.75,7.00,5.04,1.85,4.16,5856733,81.40,1.96,1.90,2.33,0.93,1.54,69727.987368
4,Denmark,2022,401945576108,9.48,5.30,4.43,7.70,7.67,5903037,81.30,1.99,1.90,2.40,0.87,1.59,68091.319114


Now the GDP per Capita is also a decimal, so I rounded it two decimals

In [9]:
world_bank_data = df_full.round(2)
world_bank_data.head()

,Country,Year,GDP Current USD,Health Expenditure % GDP,Education Expenditure % GDP,Unemployment Rate %,Inflation Annual %,FDI Net Inflows % GDP,Population Total,Life Expectancy Years,Govt Effectiveness,Rule of Law,Control of Corruption,Political Stability,Voice and Accountability,GDP Per Capita
0,Denmark,2018,355293374912,10.10,7.30,5.13,0.81,2.47,5793636,80.95,1.81,1.77,2.16,0.93,1.57,61324.77
1,Denmark,2019,345401473013,10.15,7.26,5.02,0.76,-1.10,5814422,81.45,1.87,1.83,2.12,0.97,1.54,59404.27
2,Denmark,2020,355631021932,10.72,7.38,5.64,0.42,0.36,5831404,81.60,1.84,1.81,2.24,0.92,1.51,60985.49
3,Denmark,2021,408378204640,10.75,7.00,5.04,1.85,4.16,5856733,81.40,1.96,1.90,2.33,0.93,1.54,69727.99
4,Denmark,2022,401945576108,9.48,5.30,4.43,7.70,7.67,5903037,81.30,1.99,1.90,2.40,0.87,1.59,68091.32


I then saved that to processed data so that it's stored in the project

In [10]:
world_bank_data.to_csv('/files/me204-2025-project-jayden/data/processed/world_bank_cleaned_data.csv', index=False)

## World Happiness Index Data Cleaning and Processing

Now that the full dataset of Economic Indicators is cleaned and ready to use, I will now clean, compile, and merge all of my World Happiness Data of csv's into one World Happiness Data dataframe with only the countries that I need. The raw csv's for this folder have different column values and names, so a large amount of cleaning needs to be done before this dataset is ready

In [11]:
# Retrieving the data from the files
folder_path = '/files/me204-2025-project-jayden/data/raw/World_Happiness_Data'

all_data = []

for year in range(2018, 2024):
    # Made a year counter so I can add it for every row to the dataset
    year_str = str(year)
    # Constructed the full file path for the given year's CSV
    file_path = os.path.join(folder_path, f"{year_str}.csv")

    df = pd.read_csv(file_path)
    # Retrieved the column name mapping for this specific year from the util year_column_map 
    # function, which basically is just a parameter dictionary of the columns I want to pull from the
    # Happiness Index Dataset
    col_map = year_column_map[year_str]
    
    # Used dictionary comprehension to extract and rename relevant columns
    # This function uses the year_column_map dictionary to pull the values thta I need from the World Happiness
    # Index even if columns don't have the same name as other datasets
    extracted = {
        final_col: (
            range(1, len(df) + 1) if final_col == 'Rank' and orig_col is None
            # If the final column is 'Rank' and the original column is missing, then I generated rank values 
            # from 1 to n
            else df[orig_col] if orig_col in df.columns
            else [None] * len(df)
            # If the original column existed in the DataFrame, then I pulled its values from the year_column_map
            # otherwise, if the column was missing, I filled it with None values
        )
        for final_col, orig_col in col_map.items()
        # Loop through all years/csv's
    }
    
    cleaned_df = pd.DataFrame(extracted)
    # Added a year column so we can keep track of the year
    cleaned_df['Year'] = year
    # Appended all the clean datasets all onto each other
    all_data.append(cleaned_df)

# Concatenate all years into a dataframe
final_df = pd.concat(all_data, ignore_index=True)
final_df

,Rank,Country,Happiness Score,Social support,Freedom to make life choices,Generosity,Perceptions of corruption,Year
0,1,Finland,7.632,1.592,0.681,0.202,0.393,2018
1,2,Norway,7.594,1.582,0.686,0.286,0.340,2018
2,3,Denmark,7.555,1.590,0.683,0.284,0.408,2018
3,4,Iceland,7.495,1.644,0.677,0.353,0.138,2018
4,5,Switzerland,7.487,1.549,0.660,0.256,0.357,2018
...,...,...,...,...,...,...,...,...
893,133,Congo (Kinshasa),3.207,0.652,0.664,0.086,0.834,2023
894,134,Zimbabwe,3.204,0.690,0.654,-0.046,0.766,2023
895,135,Sierra Leone,3.138,0.555,0.660,0.105,0.858,2023
896,136,Lebanon,2.392,0.530,0.474,-0.141,0.891,2023


The Year column being at the end (the last column) is not optimal, especially if I am turning this into a dataset  in a the database, and year is a key. For this reason and readability, I will reorder the columns. I also only need the 6 countries that I am analyzing.

In [12]:
# Reorder columnds
cols_order = ['Year', 'Rank', 'Country', 'Happiness Score', 'Social support', 'Freedom to make life choices', 'Generosity', 'Perceptions of corruption']
final_df = final_df[cols_order]

# I only need the 5 countries that I am analyzing
target_countries = ['United States', 'Finland', 'Sweden', 'Denmark', 'Iceland', 'Netherlands']
happiness_index_data = final_df[final_df['Country'].isin(target_countries)].reset_index(drop=True)
# Rounding the decimals
happiness_index_data = happiness_index_data.round(2)

# Save the Dataset on a csv
happiness_index_data.to_csv('/files/me204-2025-project-jayden/data/processed/merged_happiness.csv', index=False)
happiness_index_data

,Year,Rank,Country,Happiness Score,Social support,Freedom to make life choices,Generosity,Perceptions of corruption
0,2018,1,Finland,7.63,1.59,0.68,0.20,0.39
1,2018,3,Denmark,7.56,1.59,0.68,0.28,0.41
2,2018,4,Iceland,7.50,1.64,0.68,0.35,0.14
3,2018,6,Netherlands,7.44,1.49,0.64,0.33,0.30
4,2018,9,Sweden,7.31,1.50,0.66,0.28,0.38
5,2018,18,United States,6.89,1.47,0.55,0.29,0.13
6,2019,1,Finland,7.77,1.59,0.60,0.15,0.39
7,2019,2,Denmark,7.60,1.57,0.59,0.25,0.41
8,2019,4,Iceland,7.49,1.62,0.59,0.35,0.12
9,2019,5,Netherlands,7.49,1.52,0.56,0.32,0.30


Now the Happiness Index dataset is ready to use. I then created a database comprising of both the Happinass Index dataset and the World Bank Economic Indicator dataset. To do this, I used python to clean the data so it was ready for SQL, and then used SQL to create the datasetsnd used SQL Alcheny to connect it to the database.

## Processing and Creating the Database

Since SQL has its own formatting, I needed to change the data to be able to be handled by SQL. SQL doesn't accept spaces or percent symbols, so some cleaning had to be done below.

In [13]:
# Define the database path
db_path = "/files/me204-2025-project-jayden/data/database.db"

# SQL couldn't handle % and spaces, so I converted columns into a compatible form
world_bank_data.columns = (
    world_bank_data.columns
    .str.strip()
    .str.replace(" ", "_")
    # Removed spaces and replace % with percent, and space with _
    .str.replace("%", "percent")
)

happiness_index_data.columns = (
    # Did the same with the Happiness Index (cleaned it so SQL can read it)
    happiness_index_data.columns
    .str.strip()
    .str.replace(" ", "_")
)

After cleaning the dataset, and making it into a ready-to-use dataset in python, it was then time to create the dataset in SQL and connect it to the SQL database

### Creating the datasets

In [14]:
# Defined and made the schema for the world_bank_data table that I
# I will put in database.db
world_bank_schema = """
CREATE TABLE IF NOT EXISTS world_bank_data (
    Country TEXT,
    Year INTEGER,
    GDP_Current_USD INTEGER,
    Health_Expenditure_percent_GDP REAL,
    Education_Expenditure_percent_GDP REAL,
    Unemployment_Rate_percent REAL,
    Inflation_Annual_percent REAL,
    FDI_Net_Inflows_percent_GDP REAL,
    Population_Total INTEGER,
    Life_Expectancy_Years REAL,
    Govt_Effectiveness REAL,
    Rule_of_Law REAL,
    Control_of_Corruption REAL,
    Political_Stability REAL,
    Voice_and_Accountability REAL,
    GDP_Per_Capita REAL,
    FOREIGN KEY (Country, Year) REFERENCES happiness_index_data(Country, Year)
)
"""

# Defined and made the schema for happiness_index_data table that
# I will put in database.db
happiness_schema = """
CREATE TABLE IF NOT EXISTS happiness_index_data (
    Year INTEGER,
    Rank INTEGER,
    Country TEXT,
    Happiness_Score REAL,
    Social_support REAL,
    Freedom_to_make_life_choices REAL,
    Generosity REAL,
    Perceptions_of_corruption REAL,
    PRIMARY KEY (Country, Year)
)
"""


I used foreign key links to happiness data on Country and Year. These two were the only unique identifiers, but since there are multiple countries and years, the foreign key must have to match BOTH the year and Country, which is a sucessful key.

Reasoning for data types:
- Country is a text because it is a string
- Year, GDP, and Population is an integer because it is reported as a whole number
- All other values are REAL's because they have the potential to have 3 whole number values, but most have 2. They include 2 decimal values

In [15]:
# Created SQLAlchemy engine
engine = create_engine(f"sqlite:///{db_path}")

# Create tables using raw SQL
with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS world_bank_data"))
    # If the table exists then drop it
    # If the code is reran, it prevents multiple of the same column from being created
    conn.execute(text("DROP TABLE IF EXISTS happiness_index_data"))
    conn.execute(text(world_bank_schema))
    conn.execute(text(happiness_schema))
    conn.commit()

# Insert data into the tables
world_bank_data.to_sql("world_bank_data", engine, if_exists="append", index=False)
happiness_index_data.to_sql("happiness_index_data", engine, if_exists="append", index=False)

36

The database was sucessfully populated with the data from both the World Bank and the Happiness Index datasets. Now my data is ready to be used for analysis.